## SALES PREDICTION 
### EDA + xgboost model

Time Series Analysis for Sales prediction
* Baseline model made with xgboost
* feature engineering primarily done with mean encoding
* Ensembling + better feature engineering left 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
path="../input/competitive-data-science-predict-future-sales/"
items=pd.read_csv(path+"/items.csv")
item_categories=pd.read_csv(path+"/item_categories.csv")
                  
sales=pd.read_csv(path+"/sales_train.csv")
                  
shops=pd.read_csv(path+"/shops.csv")
                  
test=pd.read_csv(path+"test.csv")

print(test.head())


In [ ]:
from datetime import datetime
sales['year'] = pd.to_datetime(sales['date']).dt.strftime('%Y')
sales['month'] = pd.to_datetime(sales['date'], format='%d.%m.%Y').dt.strftime('%m') #another way for same thing

sales.head(2)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

#will make your plot outputs appear and be stored within the notebook.
%matplotlib inline 

grouped=pd.DataFrame(sales.groupby(['year','month']) ['item_cnt_day'].sum().reset_index())



sns.pointplot(x="month",y="item_cnt_day",hue="year",data=grouped)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

#will make your plot outputs appear and be stored within the notebook.
%matplotlib inline 

grouped=pd.DataFrame(sales.groupby(['year','month']) ['item_price'].sum().reset_index())



sns.pointplot(x="month",y="item_price",hue="year",data=grouped)

In [ ]:
sales=sales.query("item_price >0")

In [ ]:
print('Data set size before filter valid:', sales.shape)

sales = sales[sales['shop_id'].isin(test['shop_id'].unique())]
print(sales.head())

sales =sales[sales['item_id'].isin(test['item_id'].unique())]

In [ ]:


# Aggregate to monthly level the sales
#monthly_sales=sales.groupby(["date_block_num","shop_id","item_id"])[
    #"date_block_num","date","item_price","item_cnt_day"].agg({"date_block_num":'mean',"date":["min",'max'],"item_price":"mean","item_cnt_day":"sum"})

#monthly_sales.head(5)



Trend, seasonality and patterns
* -General trend
* -shop wise trend

In [ ]:
group_dateonly = sales.copy().groupby('date_block_num').sum()
group_dateonly['rolling mean'] = group_dateonly.item_cnt_day.rolling(window=6).mean() #6 month rolling mean
group_dateonly['rolling std'] = group_dateonly.item_cnt_day.rolling(window=6).std() #6 month rolling std

plt.figure(figsize=(15,8))
plt.plot(group_dateonly['rolling mean'], color = 'blue', label='rolling mean')
plt.plot(group_dateonly['item_cnt_day'], color = 'green', label='item count per month')
plt.plot(group_dateonly['rolling std'], color = 'red', label='rolling std')

plt.legend(loc='best')
plt.grid(True)
plt.title('general trend for items sold per month')


- as seen in years 2013 and 2014, upward trend for december (month 12 and 24)
- General trend Upwards (rolling mean + higher progressive peaks)
- Item count per month exhibits seasonality as well (peaks at month 12 and 24 + constant rolling std)

In [ ]:
def subplots_shops(df, rows, columns):
    f, axes = plt.subplots(rows, columns, figsize=(15, 24), sharex=False)
    arr = df['shop_id'].unique()
    i = 0
    for r in range(rows): 
        for c in range(columns):
            sns.lineplot(data= df[df['shop_id']==arr[i]], x='date_block_num', y='item_cnt_month', ax=axes[r,c])
            #axes[r,c].tick_params(axis='y', which='both', labelleft='off')
            axes[r,c].set_ylabel('')
            axes[r,c].set_yticklabels([])
            axes[r,c].set_yticks([])
            
            axes[r,c].set_xlabel('Shop ID:'+ str(arr[i]))
            axes[r,c].set_xticks([])
            axes[r,c].set_xticklabels([])
            i = i + 1
            
    
    #plt.title('Revenue over time period')
    plt.show()
    f.tight_layout(h_pad=5.0)
    return 

sales_groupbyshops = sales.groupby(['date_block_num', 'shop_id']).sum()
sales_groupbyshops.rename({'item_cnt_day':'item_cnt_month'},axis=1, inplace=True)
sales_groupbyshops = sales_groupbyshops.reset_index()
subplots_shops(sales_groupbyshops, 7, 6)

* -individual shops exhibiting general trend upwards 
* -individual shops exhibiting seasonality in month of december 
* -many shops exhibit the same trend + shape as exhibited in the general trend!

# Previous value benchmark (score = 1.167)

In [ ]:
#groupby shop id and item id (test data format)
previous_month = 33 # or oct2015 
previous_val_train = sales[sales['date_block_num']==33].groupby(['shop_id', 'item_id'])['item_cnt_day'].sum().copy()
previous_val_train = previous_val_train.reset_index().rename({'item_cnt_day':'item_cnt_month'}, axis=1)
previous_val_train.shape
previous_val_train.head()

In [ ]:
#merging these values with test dataframe will add item_cnt_month for submission using previous months items
print(test.shape)
submission_1 = pd.merge(previous_val_train, test,how='right', on=['shop_id', 'item_id'])

#for shops or items that werent present in both sets, filled item cnts with NaN so replaced with 0
submission_1.drop(['shop_id','item_id'], axis=1, inplace=True)
submission_1 = submission_1.sort_values(by='ID').fillna(0) 
submission_1 = submission_1.reindex(columns=['ID', 'item_cnt_month'])

#graph has outliers due to preprocessing, rarely peoply buy more than 20 items 
#submission_1['item_cnt_month'].value_counts()[20:]
submission_1['item_cnt_month'] = submission_1['item_cnt_month'].clip(0,20)

#submission_1.to_csv('previous_value_bench.csv', index=False)
#score 1.167 RMSE w/o using any ML

# Basic feature engineering (datetime values + lag values)

* merge test and sales data (test: date_block_num==34, year==2015, month=November(10))
* merge item categories
* datetime: year, month, week
* lag values
* mean encodings: 'itemcnt mean'(target) for shop id, item_id, item_category etc

In [ ]:
itemcat = pd.merge(items, item_categories, on='item_category_id')
itemcat = itemcat[['item_id', 'item_category_id']]
test = test.merge(itemcat, on='item_id')
sales = sales.merge(itemcat, on='item_id')

In [ ]:
test['date_block_num'] = 34
test['year'] = 2015
test['month'] = 11

In [ ]:
sales_group = sales.groupby(['date_block_num','shop_id','item_id','item_category_id','year', 'month']).agg({
    'item_cnt_day':'sum'}).reset_index().rename({'item_cnt_day':'item_cnt_month'}, axis=1)
#print(sales_group)

In [ ]:
#merge test and sales data for data preprocessing, feature engineering and modeling 
X_full = pd.concat([sales_group, test], ignore_index=True)
print(X_full.columns)

#test --> dateblocknum = 34
#filling up mean values for item_cnt_month for assistance with feature engineering
#no regularization (beneficial for lag values)
#model may overfit 
item_cnt_mean = sales_group.groupby('item_id')['item_cnt_month'].mean()
X_full.loc[(X_full['date_block_num']==34), 'item_cnt_month'] = X_full.loc[(X_full['date_block_num']==34), 'item_id'].map(item_cnt_mean)

#fillna with global mean
global_mean = X_full.loc[(X_full['date_block_num']<34), 'item_cnt_month'].mean()
X_full['item_cnt_month'].fillna(global_mean, inplace=True)
###item_cnt_month is to be predicted but using previous values, these can be helpful in lag variables 

In [ ]:
def lag_features(df, lags, group_cols, shift_col):
    """
    #Stackoverflow code
    Arguments:
        df (pd.DataFrame)
        lags (list((int)): the number of months to lag by
        group_cols (list(str)): the list of columns that need to be the merged key
        shift_col (str): the column name that is to be shifted by
    """

    for lag in lags:
        new_col = '{0}_lag_{1}'.format(shift_col, lag)
        df[new_col] = df.groupby(group_cols)[shift_col].shift(lag)
        df[new_col] = df[new_col].clip(0,20)
        df[new_col].fillna(0, inplace=True)
        

    return df

In [ ]:
lags = [1,2,3,4,6,12]
group_cols = ['shop_id', 'item_id']

X_full = lag_features(X_full,lags,group_cols,'item_cnt_month')

* target (== item_cnt_month) encoding
* Expanding mean scheme shop/item pairs
* KFold and LOO scheme shop/item pairs
* item_category_id mean encoding
* setup train, validation, test set

In [ ]:
#Kfold encoding target [item encode] 0.14 corr
def kfold_enc(df):
    from sklearn.model_selection import KFold
    kf = KFold(5, shuffle=False, random_state=12)
    
    #declare variable
    df['kfold_targetenc'] = np.nan
    #prevent data leakage
    data_1 = df[df['date_block_num']<33]
    data_2 = df[df['date_block_num']>=33] #test set
    
    i = data_1.columns.get_loc('kfold_targetenc')
    a = data_1.columns.get_loc('item_id')
    
    for tr_ind, val_ind in kf.split(data_1):
        item_target = data_1.iloc[tr_ind].groupby(['item_id'])['item_cnt_month'].mean()
        data_1.iloc[val_ind,i] = data_1.iloc[val_ind, a].map(item_target)
        data_2.iloc[:, i] = data_2.iloc[:, i].map(item_target)
        
    data_3 = pd.concat([data_1, data_2], ignore_index=True)
    target_mean = data_1['item_cnt_month'].mean()
    data_3['kfold_targetenc'].fillna(target_mean, inplace=True)
    corr = np.corrcoef(data_3['kfold_targetenc'], data_3['item_cnt_month'])[0][1]
    print('correlation_kfoldenc: ')
    print(corr)
    del data_1
    del data_2
    return data_3
    
X_full = kfold_enc(X_full)

In [ ]:
#Expanding mean scheme [shop/item pair]
def ems_encoding(df):
    cumsum = df.groupby(['shop_id', 'item_id'])['item_cnt_month'].cumsum()
    cumcount = df.groupby(['shop_id', 'item_id'])['item_cnt_month'].cumcount()
    target_mean = df['item_cnt_month'].mean()
    
    df['EMS_targetenc'] = (cumsum - df['item_cnt_month'])/cumcount
    df['EMS_targetenc'].fillna(target_mean, inplace=True)

    corr = np.corrcoef(df['EMS_targetenc'], df['item_cnt_month'])[0][1]
    print('expanding mean scheme correlation: ', corr)
    return df

X_full = ems_encoding(X_full)

In [ ]:
def ems_encoding_itemcat(df):
    cumsum = df.groupby(['item_category_id', 'item_id'])['item_cnt_month'].cumsum()
    cumcount = df.groupby(['item_category_id', 'item_id'])['item_cnt_month'].cumcount()
    target_mean = df['item_cnt_month'].mean()
    
    df['EMS_item_targetenc'] = (cumsum - df['item_cnt_month'])/cumcount
    df['EMS_item_targetenc'].fillna(target_mean, inplace=True)

    corr = np.corrcoef(df['EMS_item_targetenc'], df['item_cnt_month'])[0][1]
    print('expanding mean scheme correlation: ', corr)
    return df

X_full = ems_encoding_itemcat(X_full)

In [ ]:
#Leave_one_out scheme target encoding with shop/item pair 
def LOO_shopitem(df):
    df['LOO_shopitem_target'] = df.groupby(['shop_id', 'item_id'])['item_cnt_month'].transform('sum')
    n_objects = df.groupby(['shop_id', 'item_id'])['item_cnt_month'].transform('count')
    df['LOO_shopitem_target'] = (df['LOO_shopitem_target'] - df['item_cnt_month'])/(n_objects-1)
    target_mean = df['item_cnt_month'].mean()
    df['LOO_shopitem_target'].fillna(target_mean, inplace=True)
    
    corr = np.corrcoef(df['LOO_shopitem_target'], df['item_cnt_month'])[0][1]
    print('Leave_one_out scheme correlation: ', corr)
    return df
    
X_full = LOO_shopitem(X_full)

In [ ]:
def LOO_item_itemcat(df):
    df['LOO_itemcat_target'] = df.groupby(['item_category_id', 'item_id'])['item_cnt_month'].transform('sum')
    n_objects = df.groupby(['item_category_id', 'item_id'])['item_cnt_month'].transform('count')
    df['LOO_itemcat_target'] = (df['LOO_itemcat_target'] - df['item_cnt_month'])/(n_objects-1)
    target_mean = df['item_cnt_month'].mean()
    df['LOO_itemcat_target'].fillna(target_mean, inplace=True)
    
    corr = np.corrcoef(df['LOO_itemcat_target'], df['item_cnt_month'])[0][1]
    print('Leave_one_out scheme correlation: ', corr)
    return df

X_full = LOO_item_itemcat(X_full)

In [ ]:
#Smoothing scheme Target encoding item category 
#formula = (targetmean*n_rows + target_mean*alpha)/(n_rows + alpha)
#alpha used for introducing noise/smoothing/regularization
'''
def smooth_itemcat(df):
    alpha = 100 
    target_mean = df['item_cnt_month'].mean()
    
    #mean target
    itemcat_mean = df.groupby('item_category_id')['item_cnt_month'].transform('mean')
    #count of appearances
    n_rows = df.groupby('item_category_id').transform('count')
    
    df['smooth_itemcat_target'] = (itemcat_mean*n_rows + target_mean*alpha)/(n_rows + alpha)
    df['smooth_itemcat_target'].fillna(target_mean, inplace=True)
    
    del itemcat_mean
    del n_rows
    
    corr = np.corrcoef(df['smooth_itemcat_target'], df['item_cnt_month'])[0][1]
    print('Smoothing scheme correlation: ', corr)
    return df

_s = smooth_itemcat(X_full)
_s.isna().sum()
'''

In [ ]:
plt.figure(figsize=(15,8))
sns.heatmap(X_full.corr())
#target encodings + lag features (unsurprisingly) show significant correlation with item_cnt

# Baseline model with XGBoost 

In [ ]:
X_full = X_full.astype({'year':'int64', 'month':'int64'})

In [ ]:
#XGBoost model primary test
#metric rmse 
import xgboost as xgb
from xgboost import plot_importance, plot_tree
from sklearn.metrics import mean_squared_error
x_cols = ['date_block_num', 'shop_id', 'item_id', 'item_category_id', 'year',
          'month', 'kfold_targetenc', 'EMS_targetenc', 'item_cnt_month_lag_1',
          'item_cnt_month_lag_2', 'item_cnt_month_lag_3','item_cnt_month_lag_4',
          'item_cnt_month_lag_6','item_cnt_month_lag_12', 'LOO_shopitem_target', 
          'LOO_itemcat_target', 'EMS_item_targetenc']

y_train = X_full[X_full['date_block_num']<33]['item_cnt_month'].copy().fillna(0).clip(0,20)
y_valid = X_full[X_full['date_block_num']==33]['item_cnt_month'].copy().fillna(0).clip(0,20)

X_train = X_full[X_full['date_block_num']<33][x_cols].copy()
X_valid = X_full[X_full['date_block_num']==33][x_cols].copy()
X_test = X_full[X_full['date_block_num']>33][x_cols].copy()



In [ ]:
#basic model fit
#baseline model rmse=1.87
model = xgb.XGBRegressor(max_depth=8,min_child_weight=2, n_estimators=1000)
model.fit(X_train, y_train,
          eval_metric="rmse",
          eval_set=[(X_train, y_train), (X_valid, y_valid)],
          early_stopping_rounds=20,
          verbose=True)

In [ ]:
_ = plot_importance(model, height=0.9)

In [ ]:
#performance evaluation 
y_pred = model.predict(X_valid).clip(0,20)
rmse_val = mean_squared_error(y_valid,y_pred, squared=False)

print('rmse_validation: ', rmse_val)

In [ ]:
#test['item_cnt_month'] = model.predict(X_full[X_full['date_block_num']==34][x_cols]).clip(0,20)
#baseline_sub = test[['ID', 'item_cnt_month']]
#baseline_sub.to_csv('baseline_submission.csv', index=False)
#rsme 1.9

# Hyper parameter turning + Ensembling

* Tune xgboost
* Ensemble with linear + randomforest + gradient boosting (simple regularization) 
* Stacking 

In [ ]:
#min(X_full[X_full['date_block_num']==33].index)
#max(X_full[X_full['date_block_num']==33].index)

In [ ]:
#model tuning with xgboost
#y_train y_valid X_train X_valid

def parameter_tuning(xtrain, ytrain, xvalid, yvalid):
    import xgboost as xgb
    from sklearn.metrics import mean_squared_error
    
    model_t =  xgb
    
    #Manual trying different combinations
    param_grid_xgb = {
        'min_child_weight': 2,
        'gamma': 0.5,
        'colsample_bytree': 0.8,
        'max_depth': 7,
        'num_round': 30,
        'eval_metric':'rmse',
        'n_estimators': 1000
        }
    model = xgb.XGBRegressor(param_grid=param_grid_xgb)
    model.fit(xtrain, ytrain,
              eval_metric="rmse",
              eval_set=[(xtrain, ytrain), (xvalid, yvalid)],
              early_stopping_rounds=20,
              verbose=True)
    
parameter_tuning(X_train, y_train, X_valid, y_valid)
#rmse of approx 1.599 on validation set

Ensembling 

* Simple linear mix with xgboost and linear regression 
* Stacked Ensembling with Extratrees, Catboost, Lightgbm, Xgboost, Randomforrest 

In [ ]:
#Linear regression + xgb for simple mix

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score

lr = LinearRegression()
lr.fit(X_train, y_train)
pred_lr = lr.predict(X_valid)

print('Test R-squared for linreg is %f' % r2_score(y_valid, pred_lr))

'''
r_lr = Ridge(alpha=2)
r_lr.fit(X_train, y_train)
pred_r = r_lr.predict(X_valid)

print('Test R-squared for Ridgereg is %f' % r2_score(y_valid, pred_r))
#gives same score as linear regression
'''


import xgboost as xgb
param_grid_xgb = {
        'min_child_weight': 2,
        'gamma': 0.5,
        'colsample_bytree': 0.8,
        'max_depth': 7,
        'num_round': 30,
        'eval_metric':'rmse',
        'n_estimators': 1000
        }
#model = xgb.XGBRegressor(param_grid=param_grid_xgb)
xgb_model = xgb.XGBRegressor(param_grid=param_grid_xgb)
xgb_model.fit(X_train, y_train,
          eval_metric="rmse",
          eval_set=[(X_train, y_train), (X_valid, y_valid)],
          early_stopping_rounds=20,
          verbose=False)
pred_xgb = xgb_model.predict(X_valid)
print('Test R-squared for Xgbreg is %f' % r2_score(y_valid, pred_xgb))

X_valid_level2 = np.concatenate([pred_lr, pred_xgb])

In [ ]:
#X_train_level2.shape
X_float = X_full.select_dtypes('float64').columns
X_int = X_full.select_dtypes('int64').columns

X_full[X_float] = X_full[X_float].astype('float32')
X_full[X_int] = X_full[X_int].astype('int32')
to_drop_cols = ['ID', 'item_cnt_month']
dates = X_full['date_block_num']
last_block = 33 #validation month
dates_train = dates[dates <  last_block]
dates_train_level2 = dates_train[dates_train.isin([27, 28, 29, 30, 31, 32])]
#meta features 

y_train_level2 = y_train[dates_train.isin([27, 28, 29, 30, 31, 32])]
X_train_level2 = np.zeros([y_train_level2.shape[0], 2])

In [ ]:
#train and inserts meta features in 
#level 2 X

dates_train_level2 = dates_train_level2.reset_index(drop=True)
#print(dates_train_level2)
for cur_block_num in [27, 28, 29, 30, 31, 32]:
    '''
        1. Split `X_train` into parts
           Remember, that corresponding dates are stored in `dates_train` 
        2. Fit linear regression 
        3. Fit XGB and put predictions          
        4. Store predictions from 2. and 3. in the right place of `X_train_level2`. 
           You can use `dates_train_level2` for it
           Make sure the order of the meta-features is the same as in `X_test_level2`
    ''' 
    train_block = X_full.loc[dates < cur_block_num].drop(to_drop_cols, axis=1)
    test_block = X_full.loc[dates == cur_block_num].drop(to_drop_cols, axis=1)
    
    y_train_block = X_full.loc[dates <  cur_block_num, 'item_cnt_month'].values
    y_test_block = X_full.loc[dates == cur_block_num, 'item_cnt_month'].values
    
    #print(train_block.info())
    #print(cur_block_num)
    
    #metafeature1
    model_lr = LinearRegression()
    model_lr.fit(train_block, y_train_block)
    X_train_level2[dates_train_level2 == cur_block_num, 0] = model_lr.predict(test_block) 
    
    #metafeature 2
    model_xgb = xgb.XGBRegressor(param_grid = 
        {
        'min_child_weight': 2,
        'gamma': 0.5,
        'colsample_bytree': 0.8,
        'max_depth': 7,
        'num_round': 30,
        'eval_metric':'rmse',
        'n_estimators': 1000
        })
    
    model_xgb.fit(train_block, y_train_block,
                  eval_metric="rmse",
                  verbose=False)
    X_train_level2[dates_train_level2 == cur_block_num, 1] = model_xgb.predict(test_block) 

In [ ]:
plt.scatter(X_train_level2[:, 0], X_train_level2[:, 1])
plt.title('Xgb vs Linear reg Metafeature')
plt.xticks([])
plt.yticks([])

#linear reg w/ xgboost
#significant correlation, some outliers

Using a simple linear mix to ensemble:

mix = α⋅linreg_prediction+(1−α)⋅xgb_prediction